# Extending From Two-Group to Three-Group Models (COMPLETE)

## Chapter 8.1-8.3 Overview Notebook

In [ ]:
# teacher: please enter a value for class_id
class_id <- "arthur" # put any unique id here (e.g., teacher email address)
user_id <- Sys.getenv("JUPYTERHUB_USER") # no need to do anything here; this gets the users Jupyter ID

# run this to set up the notebook
library(coursekata)
library(gridExtra)

# set styles
css <- suppressWarnings(readLines("https://raw.githubusercontent.com/jimstigler/jupyter/master/ck_jupyter_styles_v2.css"))
IRdisplay::display_html(sprintf('<style>%s</style>', paste(css, collapse = "\n")))

# generate pet data
set.seed(1)
n <- 90
pet_type <- rep(c("Dog+Cat", "Dog", "Cat"), each = n / 3)
happiness <- round(c(
  rnorm(n/3, mean = 5, sd = 1.8),  # Dog+Cat
  rnorm(n/3, mean = 5.5, sd = 1.8),  # Dog
  rnorm(n/3, mean = 4.5, sd = 1.8)   # Cat
), 1)
pet_owners <- sample(data.frame(pet_type, happiness)) %>%
  select(-orig.id)
# write.csv(df, "pet_owners.csv", row.names = FALSE)


<div class="teacher-note">
    
<b>Section Goals:</b> In this section, students extend their understanding of group models from two-group to three-group models, learning how to specify the models in GLM notation, fit the models using R, and interpret parameter estimates and ANOVA tables from more complex categorical models. They deepen their understanding of model predictions, residuals, sums of squares, and model comparison using PRE in the context of multi-group models.

Students should understand that:
- A three-group model can be specified using GLM notation using two dummy-coded variables ($X_1$, $X_2$) representing membership in group 2 and group 3, respectively.
- The parameter estimates from a three-group model function similarly to the two-group model: $b_0$ is the mean of the reference group (the predicted value of Y when all Xs = 0); $b_1$ and $b_2$ represent the differences between the reference group and the other two groups.
- The following ideas are the same as in two-group models:
    - Model predictions are generated by multiplying the dummy-coded predictor variables by their parameter estimates.
    - Residuals represent the difference between observed and predicted values (or the data and model predictions, respectively). Squaring and summing these yields the Sum of Squared Error (SSE).
    - The total variation (SST) can be partitioned into SSM + SSE, and PRE can be used to indicate the proportion of total variation on the outcome variable that is explained by the model that includes an explanatory variable.
- Adding more groups to a model typically (but not always) reduces SSE and increases PRE (Proportion Reduction in Error), because having more group-specific predictions allows the model to more closely match the data.


A <a href="https://docs.google.com/document/d/1NfbdOfq6TE72Jvcf6of1eK8nC8qkavRgfffQUpFOS1A/edit?tab=t.5y2a0ykmi2fk#heading=h.1k54q7nupylh" target="_blank">printable student guided-notes worksheet</a> is available to go with this Jupyter notebook, as well as a student version of this notebook.
</div>

## 1 Happiness of Pet Owners

There are, of course, cat people and dog people. But which people are happier? And what about people with both? Believe it or not, there is a lot of research on this. But today let's just look at a small sample of data from a survey of pet owners.

<img src="https://coursekata-course-assets.s3.us-west-1.amazonaws.com/UCLATALL/czi-stats-course/cats-dogs-1.png">

### 1.1 The survey questions

The participating pet owners answered a number of questions, two of which were:

1. Do you own a pet? If so, what kind? Responses were coded as `Cat`, `Dog`, `Dog+Cat`, or `Neither`.
2. On a scale of 1 to 10, how happy are you these days? 1 = extremely unhappy; 10 = extremely happy.

Run the cell below to see what the survey looked like and try answering it.

<div class="teacher-note">

**Teacher Note:** Depending on how you are implementing this notebook, you could go ahead and let students input their own data for the survey, either because each student is following along on their own device, or by letting a few students come to the front to enter data. Still, the main part of the notebook will be devoted to analyzing existing data. Students could analyze their own class data later, in a *Practicing What You've Learned* section.

</div>


In [ ]:
IRdisplay::display_html(sprintf(
  '<iframe src="https://uclatall.github.io/jupyter-survey/cats-n-dogs-v3.html?class_id=%s&user_id=%s" width="100%%" height="520" 
    sandbox="allow-scripts allow-same-origin" 
    style="border: none; box-shadow: none; background: white; "></iframe>',
  class_id, user_id
))

### 1.2 Write some code to see what's in the `pet_owners` dataset and to explore the distributions of the variables. 

Note: A few participants reported having neither a cat nor a dog. Because that group was so small, we’ve excluded them from this dataset to keep things simple.

The `pet_owners` dataset has two variable:

- `pet_type`: `Cat`, `Dog`, `Dog+Cat`
- `happiness`: a 1-10 scale with 1=Extremely Happy, 10=Extremely Unhappy

<div class="teacher-note">
    
**Teacher Note:** 

- By now, most students know how to do basic things like examine datasets and distributions of single variables. It's a good rule-of-thumb to start any analysis by exploring the distributions of the variables. Often, this is where mistakes or misinterpretations in the data are found, which saves lots of time and frustration later. You can type this code, but ask students for suggestions.
    
- You may want to have a side discussion about the pros/cons of excluding very small groups in real research, a common practice. Students can also think about how extreme situations (e.g., 1 person in a group) might not be good for a variety of reasons (e.g., privacy, no variation, generalizability and stability of models). An alternative approach could be to combine the few "Neithers" with another group, but that would introduce new assumptions.

</div>


In [ ]:
# code here

# sample responses
head(pet_owners,10)
str(pet_owners)

gf_histogram(~ happiness, data = pet_owners, binwidth = 1)

tally(~ pet_type, data = pet_owners)

<div class="guided-notes">

### 1.3 Who do you think would be happier on average? A cat owner, a dog owner, or someone who owns both dogs and cats? Explain your hypothesis. Also write a word equation that says: if we knew a person's `pet_type`, we would be able to make a better prediction of their `happiness`. 
    
</div>


### 1.4 Write code to visualize the relationship between `pet_type` and `happiness` 

In [ ]:
# code here


# sample response
gf_jitter(happiness ~ pet_type, data = pet_owners, width = .2)


<div class="discussion-question">

### 1.5 Discussion Question: What do you think? Can you tell who is happier on average?
    
</div>

<div class="teacher-note">

**Sample Responses:**  
- The dog group might be a little happier on average than the other two groups.  
- There’s a lot of variation, so people in any pet group can feel the whole gamut of happiness (from 1 to 9 on the scale).
- Dog people don’t seem to report being extremely unhappy (scores of 1–2).

**Teacher Note:** This is a quick opportunity to make informal observations; they’ll formalize this with a best-fitting model in the next section.

</div>


## 2 Fitting and Interpreting Three-Group Model

Most of the concepts and R code you used for constructing and interpreting two-group models can be directly applied to three-group models. There are a few minor differences, but mostly, you can extend your understanding from two-group to three-group models without much trouble. Let's start with fitting the model.

<div class="guided-notes">

### 2.1 Write the code you think you would use to fit the three-group model of happiness based on `pet_type` (Cat, Dog, or Dog+Cat) in the second row of the table.
    
</div>


<table border="1" style="font-size: 18px; margin-left: 0; border-collapse: collapse; width: 100%;">
  <thead>
    <tr>
      <td style="border: 1px solid black; font-weight: bold; text-align: center; width:22%"></td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center; width:39%">Two-Group Model</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center; width:39%">Three-Group Model</td>
    </tr>
  </thead>
  <tbody>
    <tr>
        <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top;"><b>Example</b></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"><code>guess ~ condition</code></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"><code>happiness ~ pet_type</code></td>
    </tr>
    <tr>
        <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px"><b>R to Fit Model</b></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"><code>lm(guess ~ condition, data = anchor_data)</code></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"> </td>
    </tr>
  </tbody>  
</table>

### 2.2 Write code in the cell below to fit and save `pet_type_model`.  Overlay the model onto the jitter plot of `happiness` by `pet_type`

In [ ]:
# fit and save the model here

# modify this code to overlay the model onto the plot
gf_jitter(happiness ~ pet_type, data = pet_owners, width = .2)

# COMPLETE
# fit and save the model here
pet_type_model <- lm(happiness ~ pet_type, data = pet_owners)

# modify this code to overlay the model onto the plot
gf_jitter(happiness ~ pet_type, data = pet_owners, width = .2) %>%
  gf_model(pet_type_model)

<div class="discussion-question">

### 2.3 Discussion Question: Now that you see the `pet_type` model's predictions, which group seems happier on average? Any other patterns you notice?

</div>

<div class="teacher-note">

**Sample Responses:**  

- The dog owners are the most happy (on average), and cat owners the least and maybe the dog+cat owners are in the middle. 
- There is still a lot of variation within each group.
</div>


<div class="discussion-question">

### 2.4 Discussion Questions: How many unique predictions do you think the `pet_type_model` will make? What will those predictions be?   

</div>

<div class="teacher-note">

**Sample Responses:**  

- Three predictions: one for cats, one for dogs, and one for dog+cat. They will probably be each group's mean.
- I think it will make as many predictions as there are people in the dataset, since each person has a score. (This is true in the sense that `predict(pet_type_model)` will output one value per row in the data frame. Redirect students to consider how many *unique* predictions the model can make.)  
- Two predictions, because that’s what we saw before in the two-group model. (A potential misconception.)  

**Teacher Note:**  
- Connect back to the two-group case to highlight the pattern: one prediction per group. So a three-group model will make three predictions. How about a four-group model? Five? 

</div>


### 2.5 Print out the parameter estimates for a three-group model

What are the parameter estimates for `pet_type_model`?

In [ ]:
# code here

# COMPLETE
pet_type_model

# or
#lm(happiness ~ pet_type, data = pet_owners)

<div class="guided-notes">

### 2.6 The R output has three numbers labeled `(Intercept)`, `pet_typeDog`, and `pet_typeDog+Cat`. Label them as $b_0$, $b_1$, and $b_2$.
    
</div>


<div class="guided-notes">

### 2.7 Draw and write on the jitter plot to show where $b_0$, $b_1$, and $b_2$ are represented   

</div>


In [ ]:
# run this code
pet_type_model

gf_jitter(happiness ~ pet_type, data = pet_owners, width = .2) %>%
gf_model(pet_type_model)

<div class="teacher-note">

**Teacher Note:** 
- $b_0$ = the mean of the reference group (Cat owners).
- $b_1$ = the adjustment from Cat owners to Dog owners.
- $b_2$ = the adjustment from Cat owners to Dog+Cat owners.

Students are pretty good at noticing that $b_0$ is the mean of the reference group. $b_1$ is the same as the two group model. The hardest one is $b_2$. The most common misconception is that this is the difference between the 2nd and 3rd group (note to students that if this were the case, $b_2$ would be negative).

</div>


<div class="guided-notes">

### 2.8 Write in the first three rows how you would interpret each of the parameter estimates:  
- $b_0$ `(Intercept)`  
- $b_1$ `pet_typeDog`  
- $b_2$ `pet_typeDog+Cat`  

Hint: Build off what you know about interpreting $b_0$ and $b_1$ in two-group models. In the table, we put in an example of how we might interpret a two-group model that only had Cat and Dog owners. 

</div>

<table border="1" style="font-size: 18px; margin-left: 0; border-collapse: collapse; width: 100%;">
  <thead>
    <tr>
      <td style="border: 1px solid black; font-weight: bold; text-align: center; width:24%"></td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center; width:38%">Two-Group Model</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center; width:38%">Three-Group Model</td>
    </tr>
  </thead>
  <tbody>
    <tr>
        <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px">$b_0$<br><code>Intercept</code></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">mean of reference group</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
    <tr>
       <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px">$b_1$<br><code>pet_typeDog</code></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">adjustment to get from mean of reference group to mean of second group</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"> </td>
    </tr>
    <tr>
    <tr>
       <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px">$b_2$<br><code>pet_typeDog+Cat</code></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">NA</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
    <tr>
      <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px"><b>Model specification (with variable names)</b></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">$$\text{happiness}_i = b_0 + b_1\text{pet_typeDog}_i + e_i$$</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
    <tr>
      <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px"><b>Model specification (with $Y$, $X_{1i}$, $X_{2i}$)</b></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">$$Y_i = b_0 + b_1X_{1i} + e_i$$</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
    <tr>
       <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px;"><b>Residual</b></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">$$e_i = Y_i - \hat{Y}_i$$</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
  </tbody>
</table>  

## 3 How R Generates Predictions — GLM Notation for the Three-Group Model

The three-group model generates three parameter estimates. These parameter estimates are part of the function that generates the three model predictions, which are the three group means. But how do they do that? Let's look at the GLM notation for the two-group model, and see if we can figure out how it would work for the three-group model.

For example, if we only had the Cat and Dog groups, we might write a two-group model like this:<br><br>

<span style="font-size: 20px">
$$\underbrace{\text{happiness}_i}_{\text{DATA}} \;=\; \underbrace{(b_0 + b_1\text{pet_typeDog}_i)}_{\text{MODEL}} \;+\; \underbrace{e_i}_{\text{ERROR}}$$</span>

<div class="guided-notes">

### 3.1 In the next row of the large table in your guided notes, write how you think the three-group model would be written using variable names 
</div>

<div class="guided-notes">

### 3.2 In the next row of the table, write the same model in GLM notation, this time using $Y_i$ to represent the outcome variable, and $X_{1i}$ and $X_{2i}$ to represent the explanatory variables.  

</div>

<div style="font-size: 18px; line-height: 1.4; border: 2px solid black; padding: 10px;">
    
We use <b>dummy coding</b> to represent predictor variables in the General Linear Model. (There are other ways to represent predictor variables; this is just one option.) This means that regardless of how many predictor variables we have, we write them such that each $X$ can only take two values: 0 or 1. 
    
Each group (beyond the reference group) is represented with an $X_i$. If there is more than one $X_i$ we name them $X_{1i}$, $X_{2i}$, and so on. For all of the $X$s, 1 = is in that group; 0 = is not in that group.
    
</div>

### 3.3 How the model makes predictions

A model is a function that generates a prediction for every row in a dataset. The model function for the three-group model can be written like this: 

<span style="font-size: 20px">$$\hat{Y}_i=b_0+b_1X_{1i}+b_2X_{2i}$$</span>

<div class="discussion-question">

### 3.4 Discussion Questions: Why do we use $\hat{Y}$ instead of $Y_i$ in this equation? Why is there no $e_i$? 

</div>

<div class="teacher-note">

**Teacher Note:** 
$\hat{Y}_i$ represents the model's prediction for row *i* in the dataset, not the value of $Y_i$ in the data. $e_i$ is only need to account for the difference between the model prediction and the actual data. In this case, we only have the prediction.

</div>


<div class="guided-notes">
    
### 3.5 In the table below, fill in the values that $X_{1i}$ and $X_{2i}$ would have for each of the three groups. Then substitute those values into the function that makes predictions for the three-group model, and fill in the other missing cells in the table. 

</div>

<table border="1" style="font-size: 18px; margin-left: 0; border-collapse: collapse; width: 100%;">
  <thead>
    <tr>
      <td style="border: 1px solid black; font-weight: bold; text-align: left;" width = 10%>Group</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;" width = 5%>$X_{1i}$</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;" width = 5%>$X_{2i}$</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;" width = 23%>Function</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;" width = 13%>Prediction</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;">Interpretation of Prediction</td>
    </tr>
  </thead>
  <tbody>
    <tr style="height: 80px;">
      <td style="border: 1px solid black; font-weight: bold; text-align: left">Cat</td>
      <td style="border: 1px solid black; text-align: left; text-align: center;">0</td>
      <td style="border: 1px solid black; text-align: left; text-align: center;">0</td>
      <td style="border: 1px solid black; text-align: center;">$b_0+b_1(\text{__})+b_2(\text{__})$</td>
      <td style="border: 1px solid black; text-align: center;">$$\\b_0$$</td>
      <td style="border: 1px solid black; text-align: center;"> </td>
    </tr>
    <tr style="height: 80px;">
      <td style="border: 1px solid black; font-weight: bold; text-align: left">Dog</td>
        <td style="border: 1px solid black; font-weight: bold; text-align: center"></td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;"></td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;">$b_0+b_1(\ 1\ )+b_2(\ 0\ )$</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;">$$\\b_0 + b_1$$</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;"></td>
    </tr>
    <tr style="height: 80px;">
      <td style="border: 1px solid black; font-weight: bold; text-align: left">Dog+Cat</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center"></td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;"></td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;">$b_0+b_1(\text{__})+b_2(\text{__})$</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center;"></td>
      <td style="border: 1px solid black; text-align: center;">mean happiness of Dog+Cat group</td>
    </tr>
  </tbody>  
</table>

<div class="discussion-question">
    
### 3.6 Discussion Question: In the three-group pet model, what do the dummy-coded variables mean?

- What does $X_{1i}$ represent?
- What does $X_{2i}$ represent?


</div>

<div style="font-size: 18px; line-height: 1.4; border: 2px solid black; padding: 10px;">

R does not require you to set up dummy-coded variables; it does that automatically. So, if you create the <code>pet_type_model</code> using <code>lm(happiness ~ pet_type, data = pet_owners)</code>, R transforms that into a 3-group model with two dummy-coded $X$ variables.
    
</div>

<div class="discussion-question">
    
### 3.7 Discussion Question: If there was a 4-group model (e.g., Cat, Dog, Dog+Cat, Neither), how many dummy-coded $X$s would R set up?

</div>

<div class="teacher-note">

**Teacher Note:**  
- A model with *k* groups requires *k–1* dummy-coded predictors ($X$s). Thus a model with 3 groups has 2 dummy-coded $X$s and a model with 4 groups has 3 dummy-coded $X$s. 


</div>


<div class="discussion-question">

### 3.8 How would you write a 4-group model in GLM notation? What would you add to the 3-group model shown below?

<span style="font-size: 24px">$$\hat{Y}_i=b_0+b_1X_{1i}+b_2X_{2i}$$</span>

Is there a pattern between the number of $X$s and the number of groups?
    
</div>

<div class="teacher-note">

**Sample Response:**  
- We would add $b_3X_{3i}$

**Teacher Note:** 
- For 4 groups, there are four parameter estimates ($b_0, b_1, b_2, b_3$) but only three $X$s (because the first group is when all the other $X$s are 0)

</div>


## 4 Analyzing Error From the Three-Group Model

Just like the two-group model, we can assess how well our three-group model fits the data by analyzing the error around the model. Just like always, our calculations of error start with residuals. From there we move to sums of squares. And just like the two-group model, we assess model fit by comparing the SS Total (from the empty model) to the leftover SS Error (for the three-group model) to see how much error has been reduced by applying our model.

<div class="guided-notes">

### 4.1 In the final row of the table (shown below), fill in the formula used to calculate residuals from the three-group model
    
</div>


<table border="1" style="font-size: 18px; margin-left: 0; border-collapse: collapse; width: 100%;">
  <thead>
    <tr>
      <td style="border: 1px solid black; font-weight: bold; text-align: center; width:24%"></td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center; width:38%">Two-Group Model</td>
      <td style="border: 1px solid black; font-weight: bold; text-align: center; width:38%">Three-Group Model</td>
    </tr>
  </thead>
  <tbody>
    <tr>
        <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px">$b_0$<br><code>Intercept</code></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">mean of reference group</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
    <tr>
       <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px">$b_1$<br><code>pet_typeDog</code></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">adjustment to get from mean of reference group to mean of second group</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"> </td>
    </tr>
    <tr>
    <tr>
       <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px">$b_2$<br><code>pet_typeDog+Cat</code></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">NA</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
    <tr>
      <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px"><b>Model specification (with variable names)</b></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">$$\text{happiness}_i = b_0 + b_1\text{pet_typeDog}_i + e_i$$</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
    <tr>
      <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px"><b>Model specification (with $Y$, $X_{1i}$, $X_{2i}$)</b></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">$$Y_i = b_0 + b_1X_{1i} + e_i$$</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
    <tr>
       <td style="border: 1px solid black; text-align: left; width: 50%; vertical-align: top; height: 80px;"><b>Residual</b></td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;">$$e_i = Y_i - \hat{Y}_i$$</td>
      <td style="border: 1px solid black; text-align: left; vertical-align: top;"></td>
    </tr>
  </tbody>
</table>  

<div class="teacher-note">

**Teacher Note:** Residuals for any model are calculated as the data point minus the model prediction for that data point. Students should see that on the table, the formula for residuals from the three-group model is exactly the same as for the two-group model.

</div>


### 4.2 Run `supernova()` for the `pet_type_model`

In [ ]:
# code here
supernova(pet_type_model)

<div class="discussion-question">

### 4.3 Discussion Questions: How much error has been reduced from the empty model to the pet type model? What percentage of the variation in happiness is explained by pet type under this model?
    
Do you see evidence that DATA = MODEL + ERROR? Explain.
    
</div>

<div class="teacher-note">

**Sample Responses:**  
- The SS Model is around 16 (SS Error is 16 less than SS Total). 
- The PRE is 0.07, around 7% of the variation has been explained by pet type (which may not seem like much to students).
- DATA = MODEL + ERROR in the ANOVA table because SS Total equals SS Model + SS Error.
    
**Teacher Note:** Often PRE of 7% doesn't seem like much to students and that's okay for now. You can probe by asking -- "so should we not use pet type to help us predict happiness?" Students looking at the model on the graph might have the impression that pet type is a little bit helpful but just not perfect at predicting happiness because there is just so much "other stuff". That's a sophisticated understanding to land on!

</div>


## 5 Practice What You've Learned

### 5.1 Gather data from your own class and analyze it.

In [ ]:
IRdisplay::display_html(sprintf(
  '<iframe src="https://uclatall.github.io/jupyter-survey/cats-n-dogs-v3.html?class_id=%s&user_id=%s" width="100%%" height="520" 
    sandbox="allow-scripts allow-same-origin" 
    style="border: none; box-shadow: none; background: white; "></iframe>',
  class_id, user_id
))

In [ ]:
# run this code to import only data from your class
sheet_url <- "https://docs.google.com/spreadsheets/d/e/2PACX-1vTm5IafEMmLJBMdaGLiDzAsFu0lEQYXQeKJDNlPVSm33FwdoWdUjYgki1RlDQ-gVfVVH78MfEeNzozm/pub?gid=0&single=true&output=csv"
data <- suppressWarnings(utils::read.csv(sheet_url, header = TRUE, stringsAsFactors = FALSE)) %>%
  mutate(date = as.POSIXct(date, format = "%Y-%m-%dT%H:%M:%OSZ", tz = "UTC")) %>%
  filter(class_id == !!class_id) %>%
  select(date, user_id, class_id, pet_type, happiness)

head(data)

### 5.2 Make a data visualization to explore the variation in happiness based on pet type.

In [ ]:
# make a visualization here


### 5.3 Create a model and place it onto your data visualization above.

### 5.4 Write the best-fitting model in GLM notation. (Feel free to use b0, b1, X1i, X2i)

Fill in the ...

Yi = ... + ei

### 5.5 Interpret each parameter estimate.

- b0:
- b1: 
- b2:
- b3:


### 5.6 Interpret each dummy-coded variable.

- X1i: 
- X2i: 
- X3i: 
